In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
eps = np.finfo(float).eps
from numpy import log2 as log



In [ ]:
df = pd.read_csv("/content/drive/MyDrive/week3AI/play_tennis.csv")

In [ ]:
df.head()

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [ ]:
df.tail()


,outlook,temp,humidity,wind,play
9,Rain,Mild,Normal,Weak,Yes
10,Sunny,Mild,Normal,Strong,Yes
11,Overcast,Mild,High,Strong,Yes
12,Overcast,Hot,Normal,Weak,Yes
13,Rain,Mild,High,Strong,No


In [ ]:
def find_entropy(df):
    Class = df.keys()[-1]  
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy


In [ ]:
Class = df.keys()[-1]
print(Class)

play


In [ ]:
print(find_entropy)

<function find_entropy at 0x7fc652d24560>


In [ ]:
def find_entropy_attribute(df,attribute):
  Class = df.keys()[-1]
  target_variables = df[Class].unique()
  variables = df[attribute].unique()
  entropy2 = 0
  
  for variable in variables:
    entropy = 0
    for target_variable in target_variables:
      num = len(df[attribute][df[attribute]==variable][df[Class]==target_variable])
      den = len(df[attribute][df[attribute]==variable])
      fraction = num / (den+eps)
      entropy += -fraction*log(fraction+eps)
    fraction2 = den/len(df)
    entropy2 += -fraction2*entropy
  return abs(entropy2)

In [ ]:
# finding attribute with the highest imformation
def find_winner(df):
  Entropy_att=[]
  IG= []
  for key in df.keys()[:-1]: #runing for loop for all attributes
    IG.append(find_entropy(df)-find_entropy_attribute(df,key)) 
  return df.keys()[:-1][np.argmax(IG)]


In [ ]:
def get_subtable(df,node, value): #making subtable for attribute features
  return df[df[node]==value].reset_index(drop=True)

In [ ]:
def buildTree(df,tree=None):
  Class=df.keys()[-1]
  node = find_winner(df)
  attValue = np.unique(df[node])
  if tree is None:
    tree={}
    tree[node]={}
  for value in attValue:
    subtable = get_subtable(df,node,value)
    clvalue,counts = np.unique(subtable['play'],return_counts=True)

    if len(counts)==1:
      tree[node][value] = clvalue[0]
    else:
      tree[node][value] = buildTree(subtable)
  return tree

In [ ]:
print("\n Given Play Tennis Data Set:\n\n",df)
tree= buildTree(df)
import pprint
pprint.pprint(tree)



 Given Play Tennis Data Set:

      outlook  temp humidity    wind play
0      Sunny   Hot     High    Weak   No
1      Sunny   Hot     High  Strong   No
2   Overcast   Hot     High    Weak  Yes
3       Rain  Mild     High    Weak  Yes
4       Rain  Cool   Normal    Weak  Yes
5       Rain  Cool   Normal  Strong   No
6   Overcast  Cool   Normal  Strong  Yes
7      Sunny  Mild     High    Weak   No
8      Sunny  Cool   Normal    Weak  Yes
9       Rain  Mild   Normal    Weak  Yes
10     Sunny  Mild   Normal  Strong  Yes
11  Overcast  Mild     High  Strong  Yes
12  Overcast   Hot   Normal    Weak  Yes
13      Rain  Mild     High  Strong   No
{'outlook': {'Overcast': 'Yes',
             'Rain': {'wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'humidity': {'High': 'No', 'Normal': 'Yes'}}}}


In [ ]:
test={'outlook':'Sunny','temp':'Hot','humidity':'High','wind':'Weak'}
def func(test, tree, default=None):
    attribute = next(iter(tree)) 
    print(attribute) 
    if test[attribute] in tree[attribute].keys():
        print(tree[attribute].keys())
        print(test[attribute])
        result = tree[attribute][test[attribute]]
        if isinstance(result, dict):
            return func(test, result)
        else:
            return result
    else:
        return default
ans = func(test, tree)
print(ans)


outlook
dict_keys(['Overcast', 'Rain', 'Sunny'])
Sunny
humidity
dict_keys(['High', 'Normal'])
High
No
